In [1]:
using Random

module O

using Random: Random, AbstractRNG, RandomDevice

@kwdef mutable struct Xorshift64 <: AbstractRNG
    seed::UInt64 = rand(RandomDevice(), UInt64)
end

@inline getstate(xos::Xorshift64) = xos.seed

@inline function setstate!(xos::Xorshift64, seed)
    xos.seed = seed
    xos
end

struct TaskLocalXorshift64 <: AbstractRNG end

@inline getstate(::TaskLocalXorshift64) = current_task().rngState0

@inline function setstate!(x::TaskLocalXorshift64, seed)
    current_task().rngState0 = seed
    x
end

Random.seed!(rng::Union{TaskLocalXorshift64, Xorshift64}, ::Nothing) =
    setstate!(rng, rand(RandomDevice(), UInt64))

Random.seed!(rng::Union{TaskLocalXorshift64, Xorshift64}, seed) =
    setstate!(rng, seed)

@inline function Random.rand(xos::Union{TaskLocalXorshift64, Xorshift64}, ::Random.SamplerType{UInt64})
    res = getstate(xos)
    x = getstate(xos)
    x ⊻= x << 13
    x ⊻= x >> 7
    x ⊻= x << 17
    setstate!(xos, x)
    res
end

function Random.rand(xos::Union{TaskLocalXorshift64, Xorshift64},
        ::Random.SamplerTrivial{Random.CloseOpen01{Float64}})
    (rand(xos, UInt64) & UInt64(2^52 - 1)) / 2^52
end

end

@show xos = O.Xorshift64()
A = [rand(xos) for _ in 1:20]

xos = O.Xorshift64() = Main.O.Xorshift64(0x7a773b19571665b7)


20-element Vector{Float64}:
 0.4519284631590901
 0.9299239141367215
 0.1286543650887897
 0.49602294892657217
 0.5178312573713015
 0.02852614288910571
 0.2858217211553711
 0.277482916978532
 0.01861016331933385
 0.9842896313594018
 0.40843096069966767
 0.4932679147156869
 0.875161862316413
 0.5910835511766921
 0.012837342164134569
 0.5816915153769275
 0.6100355858751731
 0.7881262556610409
 0.2541660540271016
 0.25503918753414

In [2]:
@show xos = O.TaskLocalXorshift64()
A = [rand(xos) for _ in 1:20]

xos = O.TaskLocalXorshift64() = Main.O.TaskLocalXorshift64()


20-element Vector{Float64}:
 0.630010792991994
 0.9235345458998883
 0.10985381731841959
 0.368083305738681
 0.3918059821767035
 0.3471823232321527
 0.6722818515176587
 0.4715838933697729
 0.10995928019023427
 0.23877713425756264
 0.7023440119339981
 0.8336480699403321
 0.35885685843592263
 0.20119387602339733
 0.249804715095985
 0.8365464081795184
 0.40318162403397517
 0.4954764054202565
 0.7870998543847711
 0.4516960942991073